# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298297540664                   -0.85    4.8
  2   -8.300189157586       -2.72       -1.25    1.0
  3   -8.300449789288       -3.58       -1.87    3.4
  4   -8.300462897450       -4.88       -2.73    1.9
  5   -8.300464485951       -5.80       -3.18    5.8
  6   -8.300464570340       -7.07       -3.32    5.9
  7   -8.300464606635       -7.44       -3.46    1.5
  8   -8.300464627394       -7.68       -3.61    1.0
  9   -8.300464639517       -7.92       -3.79    1.0
 10   -8.300464642201       -8.57       -3.93    2.6
 11   -8.300464643917       -8.77       -4.44    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64200915700                   -0.70    5.4
  2   -16.64601676700       -2.40       -1.14    1.0
  3   -16.67915494866       -1.48       -1.86    4.5
  4   -16.67920599153       -4.29       -2.56    3.9
  5   -16.67927140472       -4.18       -2.99    3.4
  6   -16.67928501994       -4.87       -3.38    3.2
  7   -16.67928608958       -5.97       -3.74    2.4
  8   -16.67928622363       -6.87       -4.56    1.2


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32521263513                   -0.56    6.4
  2   -33.33273859972       -2.12       -0.99    1.0
  3   -33.33599042933       -2.49       -1.69    4.0
  4   -33.33690910880       -3.04       -2.49    4.2
  5   -33.33694205502       -4.48       -3.05    7.9
  6   -33.33694388629       -5.74       -3.89    8.5
  7   -33.33694392339       -7.43       -4.37    8.1


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298433135091                   -0.85    4.0
  2   -8.300281314035       -2.73       -1.59    1.0
  3   -8.300440371982       -3.80       -2.48    7.4
  4   -8.300342846851   +   -4.01       -2.21   12.5
  5   -8.300464382307       -3.92       -3.57    3.9
  6   -8.300464578668       -6.71       -3.89    4.0
  7   -8.300464639988       -7.21       -4.43    3.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32575384162                   -0.56    6.2
  2   -33.28334760741   +   -1.37       -1.25    1.0
  3   +9.719916246660   +    1.63       -0.25    9.2
  4   -33.26158962805        1.63       -1.39    7.8
  5   -33.30336133471       -1.38       -1.48    3.6
  6   -33.29846545745   +   -2.31       -1.74    3.1
  7   -32.80007788272   +   -0.30       -1.20    4.4
  8   -33.33481138667       -0.27       -2.30    4.6
  9   -33.33682634396       -2.70       -2.70    2.8
 10   -33.33670231192   +   -3.91       -2.55    3.0
 11   -33.33692896972       -3.64       -3.08    2.6
 12   -33.33693482360       -5.23       -3.16    1.5
 13   -33.33694187480       -5.15       -3.44    2.2
 14   -33.33694345293       -5.80       -3.84    4.5
 15   -33.33694343381   +   -7.72       -4.11    3.9


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.